In [ ]:
# !pip install catboost

In [ ]:
# !git clone https://github.com/fastai/fastai
# !pip install -e "fastai[dev]"

In [ ]:
from sklearn.linear_model import Lasso, LinearRegression,Ridge
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVC

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import ExtraTreeClassifier
import xgboost as xgb
from lightgbm import LGBMClassifier


# from sklearn.utils.testing import all_estimators
from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble import HistGradientBoostingClassifier, AdaBoostClassifier

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import numpy as np # linear b
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)b
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
sample_submission = pd.read_csv('../input/jobathon-may-2021-credit-card-lead-prediction/sample_submission.csv')
test =  pd.read_csv('../input/jobathon-may-2021-credit-card-lead-prediction/test.csv')
train = pd.read_csv( '../input/jobathon-may-2021-credit-card-lead-prediction/train.csv')

In [ ]:
train.head()

In [ ]:
test.head()

## Description of Dataset

- ID - IUnique Identifier for a row
- Gender- Gender of the Customer
- Age-Age of the Customer (in Years)
- Region_Code-Code of the Region for the customers
- Occupation-Occupation Type for the customer
- Channel_Code-Acquisition Channel Code for the Customer  (Encoded)
- Vintage-Vintage for the Customer (In Months)
- Credit_Product-If the Customer has any active credit product (Home loan,
Personal loan, Credit Card etc.)
- Avg_Account_Balance- Average Account Balance for the Customer in last 12 Months
- Is_Active-If the Customer is Active in last 3 Months

## No Of Records in the dataset

In [ ]:
train.info()

In [ ]:
test.info()

## get min max std deviation of the dataset /check quantiles

In [ ]:
train.describe()

In [ ]:
test.describe()

In [ ]:
train.nunique()

In [ ]:
train.head()

In [ ]:
# !pip install reg_resampler # will use if needed

In [ ]:
Target_col = 'Is_Lead'

In [ ]:
categorical_vars = ['Gender', 'Region_Code', 'Occupation', 'Channel_Code', 'Credit_Product', 'Is_Active']
conts = ['Age', 'Vintage', 'Avg_Account_Balance']

In [ ]:
train.Is_Lead.plot.hist()

In [ ]:
train.Age.plot.hist()

In [ ]:
test.Age.plot.hist()

In [ ]:
train.Avg_Account_Balance.plot.hist()

In [ ]:
test.Avg_Account_Balance.plot.hist()

In [ ]:
train.Vintage.plot.hist()

In [ ]:
test.Vintage.plot.hist()

In [ ]:
### By observing test dataset, Observed that test dont have quantity values higher than 4800 and lower than 9360
### Also the UnitPrice column have 80000 which is an outlier i consider

In [ ]:
# outliers
train = train[train['Avg_Account_Balance']<=max(test.Avg_Account_Balance)].reset_index(drop=True)

In [ ]:
## Bivariate Analysis

In [ ]:
sns.countplot(data=train,x='Region_Code', hue='Is_Lead')

In [ ]:
sns.countplot(data=train,x='Gender', hue='Is_Lead')

In [ ]:
sns.countplot(data=train,x='Occupation', hue='Is_Lead')

In [ ]:
sns.countplot(data=train,x='Channel_Code', hue='Is_Lead')

### Multivariate Analysis

In [ ]:
conts

In [ ]:
# sns.scatterplot(data=train,x='Age', y='Vintage', hue='Is_Lead')

In [ ]:
# sns.scatterplot(data=train,x='Age', y='Avg_Account_Balance', hue='Is_Lead')

In [ ]:
# sns.scatterplot(data=train,x='Vintage', y='Avg_Account_Balance', hue='Is_Lead')

In [ ]:
for i in categorical_vars:
    print(train.groupby(i)['Is_Lead'].value_counts().nlargest(8))

In [ ]:
# All the Entrepreneur have credit card interest

## Missing value analysis

In [ ]:
train.isnull().sum()

In [ ]:
train.Credit_Product = train.Credit_Product.replace({'Yes':1, 'No':0})
test.Credit_Product = test.Credit_Product.replace({'Yes':1, 'No':0})

In [ ]:
train.columns

## Feature Engeneering

In [ ]:
# from itertools import combinations
# for col1, col2 in combinations(categorical_vars,2):
#   train[col1+col2] = train[col1]+train[col2]
#   test[col1+col2] = test[col1]+test[col2]

In [ ]:
train.head()

In [ ]:
train.columns

In [ ]:
from sklearn.preprocessing import LabelEncoder
# new_cols = ['GenderRegion_Code', 'GenderOccupation',
#        'GenderChannel_Code', 'GenderCredit_Product', 'GenderIs_Active',
#        'Region_CodeOccupation', 'Region_CodeChannel_Code',
#        'Region_CodeCredit_Product', 'Region_CodeIs_Active',
#        'OccupationChannel_Code', 'OccupationCredit_Product',
#        'OccupationIs_Active', 'Channel_CodeCredit_Product',
#        'Channel_CodeIs_Active', 'Credit_ProductIs_Active']
# for col in new_cols:
#   le = LabelEncoder()
#   le.fit(train[col])
#   train[col] = le.transform(train[col])
#   test[col] = le.transform(test[col])

In [ ]:

le = LabelEncoder()
le.fit(train['Gender'])
train['Gender'] = le.transform(train['Gender'])
test['Gender'] = le.transform(test['Gender'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
ole = LabelEncoder()
ole.fit(train['Occupation'])
train['Occupation'] = ole.transform(train['Occupation'])
test['Occupation'] = ole.transform(test['Occupation'])

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# le.fit(train['Credit_Product'])
# # train['Credit_Product'] = le.transform(train['Credit_Product'])
# test['Credit_Product'] = le.transform(test['Credit_Product'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['Channel_Code'])
train['Channel_Code'] = le.transform(train['Channel_Code'])
test['Channel_Code'] = le.transform(test['Channel_Code'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['Is_Active'])
train['Is_Active'] = le.transform(train['Is_Active'])
test['Is_Active'] = le.transform(test['Is_Active'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['Region_Code'])
train['Region_Code'] = le.transform(train['Region_Code'])
test['Region_Code'] = le.transform(test['Region_Code'])

In [ ]:
# # performing freq encoding
# dicto = train['Region_Code'].value_counts(normalize=True).to_dict()
# train['Region_Code'] = train['Region_Code'].map(dicto)
# dicto = test['Region_Code'].value_counts(normalize=True).to_dict()
# test['Region_Code'] = test['Region_Code'].map(dicto)

In [ ]:
from scipy import stats

In [ ]:
# fitted_data, fitted_lambda = stats.boxcox(np.abs(train['Vintage']))
# train['Vintage_box'] = fitted_data
# fitted_data, fitted_lambda = stats.boxcox(np.abs(test['Vintage']))
# test['Vintage_box'] = fitted_data

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
est = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
est.fit(train['Age'].values.reshape(-1,1))
train['Age_quantiles'] = est.transform(train['Age'].values.reshape(-1,1)).astype(int)
test['Age_quantiles'] = est.transform(test['Age'].values.reshape(-1,1)).astype(int)

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
est = KBinsDiscretizer(n_bins=22, encode='ordinal', strategy='quantile')
est.fit(train['Vintage'].values.reshape(-1,1))
train['Vintage_quantiles'] = est.transform(train['Vintage'].values.reshape(-1,1)).astype(int)
test['Vintage_quantiles'] = est.transform(test['Vintage'].values.reshape(-1,1)).astype(int)

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
est = KBinsDiscretizer(n_bins=8, encode='ordinal', strategy='quantile')
est.fit(train['Avg_Account_Balance'].values.reshape(-1,1))
train['Avg_Account_Balance_quantiles'] = est.transform(train['Avg_Account_Balance'].values.reshape(-1,1)).astype(int)
test['Avg_Account_Balance_quantiles'] = est.transform(test['Avg_Account_Balance'].values.reshape(-1,1)).astype(int)

In [ ]:
train['Avg_Account_Balance_log'] = np.log(train['Avg_Account_Balance'])
test['Avg_Account_Balance_log'] = np.log(test['Avg_Account_Balance'])

train['Vintage_log'] = np.log(train['Vintage'])
test['Vintage_log'] = np.log(test['Vintage'])

In [ ]:
train.columns

In [ ]:

# train_missing = pd.concat([train.dropna().reset_index(drop=True)[missing_val_cols],test.dropna().reset_index(drop=True)[missing_val_cols]])

# train_missing_y = pd.concat([train[~train.Credit_Product.isnull()][target_missing],test[~test.Credit_Product.isnull()][target_missing]])

# test_data_missing = train[train.Credit_Product.isnull()][missing_val_cols]

# test_test_data_missing = test[test.Credit_Product.isnull()][missing_val_cols]

# from sklearn.experimental import enable_hist_gradient_boosting 
# from sklearn.ensemble import HistGradientBoostingClassifier, AdaBoostClassifier

# hist_model_missing = HistGradientBoostingClassifier(random_state=42)
# hist_model_missing.fit(train_missing, train_missing_y)


# test_data_missing[target_missing] = hist_model_missing.predict(test_data_missing)
# test_test_data_missing[target_missing] = hist_model_missing.predict(test_test_data_missing)
# train.loc[test_data_missing[target_missing].index,'Credit_Product'] = test_data_missing[target_missing].values
# test.loc[test_test_data_missing[target_missing].index,'Credit_Product'] = test_test_data_missing[target_missing].values

In [ ]:
# for cat in categorical_vars:
#   for cont in conts:
#     for aggs in ['mean', 'std', 'median', 'sum', 'count', 'min', 'max']:
#       train[str(cat)+'_'+str(cont)+'_'+aggs] = train[cat].replace(train.groupby(cat).aggregate({cont: aggs}).to_dict()[cont])
#       test[str(cat)+'_'+str(cont)+'_'+aggs] = test[cat].replace(test.groupby(cat).aggregate({cont: aggs}).to_dict()[cont])

## Modelling part

In [ ]:
train.columns

In [ ]:
selected_columns = ['Gender', 'Age', 'Region_Code', 'Occupation', 'Channel_Code',
       'Vintage_log', 'Credit_Product', 'Avg_Account_Balance_log', 'Is_Active']
train = train.fillna(2)
test = test.fillna(2)
X = train[selected_columns]
y = train[Target_col].values
X_test = test[selected_columns]

new_x = train[selected_columns+[Target_col]]
# from sklearn.preprocessing import MaxAbsScaler

# transformer = MaxAbsScaler().fit(X)
# X = transformer.transform(X)
# X_test = transformer.transform(X_test)


In [ ]:
from fastai.tabular.all import *
from fastai.tabular import *


In [ ]:
cat_names = ['Gender', 'Region_Code' , 'Occupation', 'Channel_Code', 'Credit_Product','Is_Active']
cont_names = ['Age', 'Vintage_log', 'Avg_Account_Balance_log']
procs = [Categorify, FillMissing, Normalize]

In [ ]:
def roc_auc(y_hat, y):
  # print(y_hat.shape, y.shape)
  return torch.tensor(roc_auc_score( y.cpu(), y_hat.cpu()),device='cuda:0')

In [ ]:
# !pip install --upgrade pandas

In [ ]:
# predictions = np.zeros(test.shape[0])
predictions = np.zeros(X_test.shape[0])
oobs = np.zeros(train.shape[0])
y = train[Target_col].values
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for i, (train_id, valid_id) in enumerate(skf.split(X,y)):
  X_valid= X.iloc[valid_id]
  ssd = IndexSplitter(list(valid_id))
  to = TabularPandas(new_x, procs=procs,
                    cat_names =cat_names,
                    cont_names = cont_names,
                    y_names='Is_Lead',
                    splits=ssd(X.index))
  dls = to.dataloaders(bs=64)
  learn = tabular_learner(dls, metrics=roc_auc)
  
  learn.fit_one_cycle(10,cbs=[SaveModelCallback(monitor='roc_auc', fname='shri')])
  learn.load('./shri')
  valid_dl = learn.dls.test_dl(X_valid)
  oobs[valid_id] = np.ravel(learn.get_preds(dl=valid_dl)[0])
  test_dl = learn.dls.test_dl(X_test)
  predictions += np.ravel(learn.get_preds(dl=test_dl)[0])

In [ ]:
roc_auc = roc_auc_score(y,oobs)
print(roc_auc)
finals = predictions/5
fastai_preds = finals
sample_submission[Target_col] = fastai_preds
sample_submission.to_csv('fastai.csv', index=False)
sample_submission[Target_col] = fastai_preds